In [1]:
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import dash_table
import plotly.express as px
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd

# Import base64 for encoding of image
import base64

# Load data from the provided CSV file
df = pd.read_csv('/home/jackcoster_snhu/Desktop/aac_shelter_outcomes.csv')

# Debug: check if dataframe is created successfully
print("Dataframe created:", df.head())
print("Dataframe schema:", df.dtypes)

#########################
# Dashboard Layout / View
#########################
# create dash application

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Debug: ensure image is loaded
print("Image encoded")

# Minimal layout for debugging
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Jack Coster Project 2 Dashboard'))),
    html.Hr(),
    html.Img(id='customer-image', src='data:image/png;base64,{}'.format(encoded_image.decode()), alt='customer image'),
    html.Div(dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
    ),
    html.Hr(),
    # Simplified DataTable
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    # Simplified row layout
    html.Div(className='row', style={'display' : 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Debug: ensure layout is set
print("Layout set")

#############################################
# Interaction Between Components / Controller
#############################################

# Radio button filters
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    print("Filter type selected:", filter_type)  # Debug: print selected filter type
    # Water Rescue filter
    if filter_type == 'water':
        df_filtered = df[(df['animal_type'] == "Dog") &
                         (df['breed'].isin(["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"])) &
                         (df['sex_upon_outcome'] == "Intact Female") &
                         (df['age_upon_outcome_in_weeks'].between(26, 156))]
    # Mountain or Wilderness Rescue filter
    elif filter_type == 'mount':
        df_filtered = df[(df['animal_type'] == "Dog") &
                         (df['breed'].isin(["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"])) &
                         (df['sex_upon_outcome'] == "Intact Male") &
                         (df['age_upon_outcome_in_weeks'].between(26, 156))]
    # Disaster Rescue or Individual Tracking filter
    elif filter_type == 'disaster':
        df_filtered = df[(df['animal_type'] == "Dog") &
                         (df['breed'].isin(["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"])) &
                         (df['sex_upon_outcome'] == "Intact Male") &
                         (df['age_upon_outcome_in_weeks'].between(20, 300))]
    # Reset to no filter
    else:
        df_filtered = df

    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df_filtered.columns]
    data = df_filtered.to_dict('records')
    
    # Debug: check if data and columns are correct
    print("Dashboard updated with filter:", filter_type)
    print("Data size:", len(data))
    
    return data, columns

# Change background color of selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    print("Updating graphs")  # Debug: indicate graph update
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return [html.Div("No data to display")]
    return [
        dcc.Graph(
            figure = px.histogram(dff, x='breed', title='Distribution of Breeds')
        )    
    ]

# Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')]
)
def update_map(viewData, row_ids):
    print("Updating map")  # Debug: indicate map update
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty or not row_ids:
        return [html.Div("No data to display")]
    row_id = row_ids[-1]  # Get the last selected row ID
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row_id]['location_lat'], dff.iloc[row_id]['location_long']], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row_id]['location_lat'], dff.iloc[row_id]['location_long']], children=[
                dl.Tooltip(dff.iloc[row_id]['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row_id]['name'])
                ])
            ])
        ])
    ]

if __name__ == '__main__':
    app.run_server(debug=True)


Dataframe created:    Unnamed: 0 age_upon_outcome animal_id animal_type                    breed  \
0           1          3 years   A746874         Cat   Domestic Shorthair Mix   
1           2           1 year   A725717         Cat   Domestic Shorthair Mix   
2           3          2 years   A716330         Dog  Chihuahua Shorthair Mix   
3           4         7 months   A733653         Cat              Siamese Mix   
4           5          2 years   A691584         Dog   Labrador Retriever Mix   

          color date_of_birth             datetime            monthyear  \
0   Black/White    2014-04-10  2017-04-11 09:00:00  2017-04-11T09:00:00   
1  Silver Tabby    2015-05-02  2016-05-06 10:49:00  2016-05-06T10:49:00   
2   Brown/White    2013-11-18  2015-12-28 18:43:00  2015-12-28T18:43:00   
3    Seal Point    2016-01-25  2016-08-27 18:11:00  2016-08-27T18:11:00   
4     Tan/White    2012-11-06  2015-05-30 13:48:00  2015-05-30T13:48:00   

    name outcome_subtype     outcome_type s